# Run multiple rainfall events
This example shows how to run multiple rainfall events. The basic workflow is:
1. Explore the rainfall boundary conditions of a dfs0 file using MIKE IO.
2. Create a boundary condition for each rainfall event. Set them all as inactive by default.
3. Run the model for each event by activating the relevant boundary condition and updating the scenario name.

#### Start with some imports we'll need

In [1]:
import clr
import System
import mikeio
from mikeplus import DataTableAccess
from mikeplus.engines import Engine1D
from pathlib import Path

#### 1. Explore rainfall boundary dfs0 file

In [2]:
# Let's start by opening the dfs0 file that contains our rainfall events.
# This file contains nine events with varying frequencies.
# Our goal is to run the model for every single one of these events.
rainfall_timeseries = Path("../tests/testdata/Db/Sirius/Sirius_IDF_rainfall.dfs0")
ds = mikeio.read(rainfall_timeseries)
ds

<mikeio.Dataset>
dims: (time:22)
time: 2019-01-01 00:00:00 - 2019-01-02 00:00:00 (22 non-equidistant records)
geometry: GeometryUndefined()
items:
  0:   F=20 <Rainfall Intensity> (mm per hour) - 3
  1:   F=10 <Rainfall Intensity> (mm per hour) - 3
  2:   F=5 <Rainfall Intensity> (mm per hour) - 3
  3:   F=2 <Rainfall Intensity> (mm per hour) - 3
  4:   F=1 <Rainfall Intensity> (mm per hour) - 3
  5:   F=0.5 <Rainfall Intensity> (mm per hour) - 3
  6:   F=0.2 <Rainfall Intensity> (mm per hour) - 3
  7:   F=0.1 <Rainfall Intensity> (mm per hour) - 3
  8:   F=0.05 <Rainfall Intensity> (mm per hour) - 3

#### 2. Create boundary conditions for each rainfall event

In [3]:
# First we open the MIKE+ database using MIKE+Py.
db_path = Path("../tests/testdata//Db/Sirius/Sirius.sqlite")
dta = DataTableAccess(db_path)
dta.open_database()
dta.is_database_open()

True

In [4]:
# We'll need the dfs0 file path, the item name, and data type. 
# This is the same info required to add boundary conditions in MIKE+.
event_info = []
for item in ds.items:
    event_info.append(
        {
                         # The dfs0 path should be relative to the .sqlite path
            "file_path": str(rainfall_timeseries.relative_to(db_path.parent)), 
            "item_name": item.name,
            "data_type": repr(item.type),
        }
    )

# Peak at the first two event infos created
event_info

[{'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=20',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=10',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=5',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=2',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=1',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.5',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.2',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.1',
  'data_type': 'Rainfall Intensity'},
 {'file_path': 'Sirius_IDF_rainfall.dfs0',
  'item_name': ' F=0.05',
  'data_type': 'Rainfall Intensity'}]

In [5]:
# Now let's create a boundary condition for each event using this info.
boundary_muids = []
for event in event_info:
    values = dict(
        VariationNo = System.Nullable[int](3), # set temporal variation to timeseries
        FlowTypeNo = System.Nullable[int](5), # set boundary condition type to Rainfall
        LoadTypeNo = System.Nullable[int](1), # set load type to stormwater runoff
        TSConnection = event['file_path'],
        TimeseriesName = event['item_name'],
        DataTypeName = event['data_type'],
    )
    muid = "Rainfall " + event['item_name']
    dta.insert("msm_BBoundary", muid, values)
    dta.set_value("msm_BBoundary", muid, "ApplyBoundaryNo", System.Nullable[int](0)) # Set boundary as inactive
    boundary_muids.append(muid)

# Peek at the first 3 boundary MUIDs created
# Hint: check out if they've been created in MIKE+ by clicking the 'Clear' button in the 'Boundary conditions' table.
boundary_muids

['Rainfall  F=20',
 'Rainfall  F=10',
 'Rainfall  F=5',
 'Rainfall  F=2',
 'Rainfall  F=1',
 'Rainfall  F=0.5',
 'Rainfall  F=0.2',
 'Rainfall  F=0.1',
 'Rainfall  F=0.05']

#### 3. Run the model for each event

In [6]:
# Create the Engine1D object that will be used to run MIKE 1D simulations.
engine = Engine1D(dta.datatables)

In [7]:
# Confirm which simulation is marked as the active one
active_simulation = dta.get_muid_where("msm_Project", "ActiveProject == 1")[0]
active_simulation

'Sirius_1_DEMO'

In [8]:
# Create a little helper function to rename the project simulation
def rename_simulation(current_muid, new_muid):
    # Rename the simulation
    dta.set_value("msm_Project", current_muid, "MUID", new_muid)

    # Change simulation MUID in the msm_ProjectOutput table too (will be fixed in future to happen automatically)
    output_muids = dta.get_muid_where("msm_ProjectOutput", f"simulationid == '{current_muid}'")
    for muid in output_muids:
        dta.set_value("msm_ProjectOutput", muid, current_muid, new_muid)

In [9]:
# Run the simulations
current_simulation_muid = active_simulation
for boundary_muid in boundary_muids:
    dta.set_value("msm_BBoundary", boundary_muid, "ApplyBoundaryNo", System.Nullable[int](1)) # Set boundary as active
    new_simulation_muid = "Simulation " + boundary_muid
    rename_simulation(current_simulation_muid, new_simulation_muid)
    engine.run(new_simulation_muid)
    current_simulation_muid = new_simulation_muid
    dta.set_value("msm_BBoundary", boundary_muid, "ApplyBoundaryNo", System.Nullable[int](0)) # Set boundary as inactive again

Simulation is started. Simulation id is 'Simulation Rainfall  F=20'
2024-02-01 16:05:42: MIKE 1D Engine started (2024 - 64 bit, .NETFramework,Version=v4.7.2 (.NET Framework 4.8.9181.0))

                     Version of MIKE 1D                            : 22 (22.0.3.17314, 2023-11-10)

                     Version of DHI.Mike1D.Engine                  : 22 (22.0.3.17314, 2023-11-10 19:29:25)

                     Version of DHI.Mike1D.RainfallRunoffModule    : 22 (22.0.3.17314, 2023-11-10 19:32:02)

2024-02-01 16:05:42: Loading setup file Sirius.sqlite...

2024-02-01 16:05:42: Diagnostics messages during load:

2024-02-01 16:05:42: WARNING: Found duplicate digipoints, or digipoints very close to each other. Removing 'duplicate' digipoint, reach 'Link_29', chainage 2.4489. (BR_WAR_DuplicateDigipoints)

2024-02-01 16:05:42: WARNING: The length of reach 'Link_29' is only 2.4600 m. The minimum reach length is 10.000 m. The length of reach 'Link_29' has been set to 10.000 m. (MU_WAR_ShortRe

In [10]:
# Check that the result files for each simulation have been created
results = [p for p in db_path.parent.glob(f"*Network_HD.res1d")]
for r in results:
    print(r)

..\tests\testdata\Db\Sirius\Simulation Rainfall  F=0.05BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=0.1BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=0.2BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=0.5BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=10BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=1BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=20BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=2BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Simulation Rainfall  F=5BaseDefault_Network_HD.res1d
..\tests\testdata\Db\Sirius\Sirius_1_DEMOBaseDefault_Network_HD.res1d
